# Pipeline to run all files
* ~~All scripts currently output Excel files because we were working independently~~
* ~~<b>We'll be changing this so that each script passes dataframes until the end of the pipeline.</b>~~
* Our plan is to not store anything (refresh daily) but need to consider how to deal with taking in user feedback
* Baseline

In [22]:
#Gather Articles
import NewsAPI as news
import os

#delete any old sorted article files
#if os.path.exists("articles.csv"):
#    os.remove("articles.csv")

#inputs in order: set manual date (yes=1/no=0, no = last 24 hours of news), manual start date as "YYYY-MM-DD"(leave empty if 0), manual end date (leave empty if 0, companies to pull (6= all 19 companies)
articleDB = news.main(1,"2019-03-11","2019-03-15",6) #output is called 'NewsAPIOutput.xlsx' in Python Scripts > Data folder

Gathering articles on (Gap Inc) OR (Foot Locker) OR (L Brands) OR Macerich OR Kimco OR TJX OR CVS OR (Home Depot) OR (Best Buy) OR (Lowe's) OR Walmart OR (Target's) OR TGT OR Amazon OR Kroger OR Walgreens OR Kohl's OR (Dollar General) OR (Bed Bath and Beyond) OR Safeway from: 2019-03-11 to 2019-03-15
797


In [23]:
#Article Cleaning (must pip install tqdm first (only once) to run)
import dataClean as dc
articleDB = dc.DataClean(articleDB)

In [24]:
#Feature Selection and Binary Article Encoding
import FeatureEncoding as fe
contentBinaryMatrix = fe.encoding(0, df=articleDB, text_col='content', norm='baseline')
titleBinaryMatrix = fe.encoding(0, df=articleDB, text_col = 'title', norm='baseline')

content
baseline
Binary Encoding
True
title
baseline
Binary Encoding
True


In [25]:
#Logistic Regression Classifier + Article Ranking, complete final file is called 'results_encoding.xlsx'
import logReg as lr
articleDB = lr.runLogReg(titleBinaryMatrix, contentBinaryMatrix, articleDB)
#articleDB = runLogReg(titleBinaryMatrix, contentBinaryMatrix, articleDB)

Index([0, 1, 'article_id', 'prediction'], dtype='object')
                                               title  \
0  Amazon’s Alexa has 80,000 Apps -- and Not a Si...   
1  The Daily 202: Russia may experiment with new ...   
2  Toilet rolls and painkillers: Britons stock up...   
3  The Health 202: Trump administration heads to ...   
4  Apple's App Store fees are coming under increa...   

                                         description  
0  Amazon counts some 80,000 “skills”—its name fo...  
1  European races this year could offer a taste o...  
2  Toilet rolls and painkillers are some of items...  
3  Its prospects for success don’t look great at ...  
4  Apple could lose billions of dollars in revenu...  


In [26]:
# This code extracts and ranks "tags" + counts frequency of tag mentions in articles 
"""
Inputs: 
    1) articleDB - uses column 'content'
    2) (optional) - tag type (could be 'ngrams'{unlimited}, 'bigrams'{terms with up to 2 words}, or 'unigrams'{single terms})
        - default is 'bigrams'
        - future work: add in noun phrases, named entities 
Outputs:
    1) articleDB = articleDB with appended columns "tags" and "tags_top_5"
    2) trendingTermsDB = keyterms by # article mentions
"""
import ContextExtraction as ce
articleDB, trendingTermsDB = ce.retrieveContext(articleDB)

100%|████████████████████████████████████████████████████████████████████████████████| 509/509 [03:24<00:00,  2.60it/s]


In [27]:
import EncodingforRecommender as rec
articleDB = rec.recommender(articleDB)

tifidf Encoding
bin Encoding


100%|███████████████████████████████████████████████████████████████████████████████| 509/509 [00:01<00:00, 343.21it/s]


tf Encoding


100%|███████████████████████████████████████████████████████████████████████████████| 509/509 [00:01<00:00, 339.68it/s]


        kevin  deakin  colleagues    online  trivia   startup    gotten  \
0    0.052632     3.0        0.10  0.011494     1.0  0.222222  0.117647   
1    0.000000     0.0        0.00  0.000000     0.0  0.000000  0.176471   
2    0.000000     0.0        0.00  0.011494     0.0  0.000000  0.000000   
3    0.000000     0.0        0.00  0.011494     0.0  0.000000  0.000000   
4    0.000000     0.0        0.00  0.000000     0.0  0.000000  0.000000   
5    0.000000     0.0        0.00  0.034483     0.0  0.000000  0.000000   
6    0.000000     0.0        0.00  0.000000     0.0  0.000000  0.000000   
7    0.000000     0.0        0.00  0.022989     0.0  0.000000  0.000000   
8    0.000000     0.0        0.00  0.000000     0.0  0.000000  0.000000   
9    0.000000     0.0        0.00  0.022989     0.0  0.000000  0.000000   
10   0.000000     0.0        0.00  0.000000     0.0  0.000000  0.000000   
11   0.000000     0.0        0.00  0.000000     0.0  0.000000  0.000000   
12   0.000000     0.0    

[509 rows x 21216 columns]


In [28]:
import frontPage as fp
frontpage = fp.FrontPage(articleDB, trendingTermsDB)

In [29]:
import pandas as pd
articles = pd.DataFrame(frontpage['articles'])
topterms= pd.DataFrame(frontpage['topterms'])
articles.columns

Index(['date', 'origContent', 'related_articles', 'source', 'tags', 'title'], dtype='object')

In [30]:
titles = pd.DataFrame(articles['tags'])

In [31]:
pd.DataFrame.to_excel(articleDB, 'Main Version-11-15.xlsx')